In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: AutoML training tabular regression model for online prediction using BigQuery

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use the Vertex SDK to create tabular regression models and do online prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

### Dataset

The dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). The version of the dataset you use only the fields year, month and day to predict the value of mean daily temperature (mean_temp).

### Objective

In this tutorial, you create an AutoML tabular regression model and deploy for online prediction from a Python script using the Vertex SDK. You can alternatively create and deploy models using the `gcloud` command-line tool or online using the Cloud Console.

The steps performed include:

- Create a Vertex `Dataset` resource.
- Train the model.
- View the model evaluation.
- Deploy the `Model` resource to a serving `Endpoint` resource.
- Make a prediction.
- Undeploy the `Model`.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

If you are using Colab or Google Cloud Notebooks, your environment already meets all the requirements to run this notebook. You can skip this step.

Otherwise, make sure your environment meets this notebook's requirements. You need the following:

- The Cloud Storage SDK
- Git
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Install the latest version of Vertex SDK for Python.

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *google-cloud-storage* library as well.

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

Install the latest GA version of *google-cloud-bigquery* library as well.

In [ ]:
! pip3 install -U google-cloud-bigquery $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
# TODO
PROJECT_ID = "python-docs-samples-tests"

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

In [3]:
# TODO
REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [119]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [5]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [6]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://python-docs-samples-testsaip-20220225133124/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [8]:
! gsutil ls -al $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [9]:
import google.cloud.aiplatform as aip

## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [45]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME, location=REGION)

# Tutorial

Now you are ready to start creating your own AutoML tabular regression model.

#### Location of BigQuery training data.

Now set the variable `IMPORT_FILE` to the location of the data table in BigQuery.

In [46]:
IMPORT_FILE = "bigquery-public-data.samples.gsod"

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `bq_source`: Alternatively, import data items from a BigQuery table into the `Dataset` resource.

This operation may take several minutes.

In [12]:
dataset = aip.TabularDataset.create(
    display_name="NOAA historical weather data" + "_" + TIMESTAMP,
    bq_source=[f"bq://{IMPORT_FILE}"],
)

label_column = "mean_temp"

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/1012616486416/locations/us-central1/datasets/1337379973329059840/operations/1914949651828572160
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/1012616486416/locations/us-central1/datasets/1337379973329059840
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/1012616486416/locations/us-central1/datasets/1337379973329059840')
projects/1012616486416/locations/us-central1/datasets/1337379973329059840


In [13]:
COLUMN_SPECS = {
    "year": "auto",
    "month": "auto",
    "day": "auto",
}

label_column = "mean_temp"

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLTabularTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `optimization_prediction_type`: The type task to train the model for.
  - `classification`: A tabuar classification model.
  - `regression`: A tabular regression model.
- `column_transformations`: (Optional): Transformations to apply to the input columns
- `optimization_objective`: The optimization objective to minimize or maximize.
  - binary classification:
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - multi-class classification:
    - `minimize-log-loss`
  - regression:
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

The instantiated object is the DAG (directed acyclic graph) for the training pipeline.

In [18]:
training_job = aip.AutoMLTabularTrainingJob(
    display_name="gsod_" + TIMESTAMP,
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

print(training_job)

#### Run the training pipeline

Next, you run the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `disable_early_stopping`: If `True`, training maybe completed before using the entire budget if the service believes it cannot further improve on the model objective measurements.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto 20 minutes.

In [19]:
model = training_job.run(
    dataset=dataset,
    model_display_name="gsod_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
    target_column=label_column,
)

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7371098464819609600?project=1012616486416
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/1012616486416/locations/us-central1/trainingPipelines/7371098464819609600 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/1012616486416/locations/us-central1/trainingPipelines/7371098464819609600 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/1012616486416/locations/us-central1/trainingPipelines/7371098464819609600 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/1012616486416/locations/us-central1/trainingPipelines/7371098464819609600 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.

## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

First, you need to get a reference to the new model. As with datasets, you can either use the reference to the model variable you created when you deployed the model or you can list all of the models in your project.

In [20]:
# Get model resource ID
models = aip.Model.list(filter="display_name=gsod_" + TIMESTAMP)

# Get a reference to the Model Service bq_client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

name: "projects/1012616486416/locations/us-central1/models/199534972162277376/evaluations/2771049630529203828"
metrics_schema_uri: "gs://google-cloud-aiplatform/schema/modelevaluation/regression_metrics_1.0.0.yaml"
metrics {
  struct_value {
    fields {
      key: "meanAbsoluteError"
      value {
        number_value: 15.619907
      }
    }
    fields {
      key: "meanAbsolutePercentageError"
      value {
        number_value: 10164985.0
      }
    }
    fields {
      key: "rSquared"
      value {
        number_value: 0.26661912
      }
    }
    fields {
      key: "rootMeanSquaredError"
      value {
        number_value: 20.74038
      }
    }
    fields {
      key: "rootMeanSquaredLogError"
      value {
        number_value: 0.84635335
      }
    }
  }
}
create_time {
  seconds: 1645843054
  nanos: 101656000
}
slice_dimensions: "annotationSpec"
model_explanation {
  mean_attributions {
    feature_attributions {
      struct_value {
        fields {
          key: "day"


## Send a batch prediction request

Now that your Model resource is trained, you can make a batch prediction. 

### Prepare the batch prediction data

First, create an artificial test dataset. For demonstration purposes, just take a random subset of the original training data.

In [99]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID, location="US", credentials=aip.initializer.global_config.credentials)

In [120]:
# Create test dataset in default region
PREDICTION_INPUT_DATASET_ID = f"gsod_prediction_{TIMESTAMP}"
dataset = bigquery.Dataset(f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}")
dataset = bq_client.create_dataset(dataset)
print(f"Created dataset {bq_client.project}.{dataset.dataset_id}")

# Create results dataset in default region
RESULTS_DATASET_ID = f"gsod_results_{TIMESTAMP}"
dataset = bigquery.Dataset(f"{PROJECT_ID}.{RESULTS_DATASET_ID}")
dataset = bq_client.create_dataset(dataset)
print(f"Created dataset {bq_client.project}.{dataset.dataset_id}")

Created dataset python-docs-samples-tests.gsod_prediction_20220226015715
Created dataset python-docs-samples-tests.gsod_results_20220226015715


In [122]:
# Randomly select 0.1% of the original dataset
query = f"""
        SELECT *
        FROM {IMPORT_FILE}
        WHERE RAND() < 0.001 
        """

PREDICTION_INPUT_TABLE_ID = f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}.test"
job_config = bigquery.QueryJobConfig(destination=PREDICTION_INPUT_TABLE_ID)

query_job = bq_client.query(query, job_config=job_config)  # API request
query_job.result()  # Waits for query to finish

Conflict: 409 Already Exists: Table python-docs-samples-tests:gsod_prediction_20220226015715.test

(job ID: 203b7902-e678-4aa1-8df2-490c6774175e)

          -----Query Job SQL Follows-----          

    |    .    |    .    |    .    |    .    |
   1:
   2:        SELECT *
   3:        FROM bigquery-public-data.samples.gsod
   4:        WHERE RAND() < 0.001 
   5:        
    |    .    |    .    |    .    |    .    |

### Make the batch prediction request

You can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `instances_format`: The format for the input instances, either 'bigquery', 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `predictions_format`: The format for the output predictions, either 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [123]:
# Note: The bigquery_source and bigquery_destination_prefix must be in the same region
PREDICTION_RESULTS_TABLE_ID = {PROJECT_ID}.{RESULTS_DATASET_ID}

batch_predict_job = model.batch_predict(
    job_display_name="tabular_regression_batch_predict_job",
    bigquery_source=f"bq://{PREDICTION_INPUT_TABLE_ID}",
    instances_format="bigquery",
    predictions_format="bigquery",
    bigquery_destination_prefix=f"bq://{PREDICTION_RESULTS_TABLE_ID}",
)

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/1012616486416/locations/us-central1/batchPredictionJobs/4229134037286912000
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/1012616486416/locations/us-central1/batchPredictionJobs/4229134037286912000')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/4229134037286912000?project=1012616486416
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1012616486416/locations/us-central1/batchPredictionJobs/4229134037286912000 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/1012616486416/locations/us-central1/batchPredictionJobs/4229134037286912000 current state:
JobState.JOB

###  View the batch prediction results

Use the BigQuery Python client to query the destination table and return results as a Pandas dataframe.

In [82]:
dataframe = (
    bq_client.query(f"SELECT * FROM {PREDICTION_RESULTS_TABLE_ID}*")
    .result()
    .to_dataframe()
)

print(dataframe.head())

   station_number  wban_number  year  month  day  mean_temp  \
0          703605        99999  1943      4   11  30.700001   
1          949950        99999  1950      1    6  75.300003   
2          999999        24223  1950      3    7  37.500000   
3          999999        14816  1950      9   16  60.700001   
4          313000        99999  1951      7   15  65.699997   

   num_mean_temp_samples  mean_dew_point  num_mean_dew_point_samples  \
0                     19       23.500000                        19.0   
1                     13       63.400002                        13.0   
2                     24       33.900002                        24.0   
3                     24       49.799999                        24.0   
4                      8       61.200001                         8.0   

   mean_sealevel_pressure  ...  min_temperature  min_temperature_explicit  \
0                     NaN  ...              NaN                      None   
1                     NaN  ...    

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Model
- AutoML Training Job
- Batch Job
- Cloud Storage Bucket

In [ ]:
# Delete BigQuery datasets
bq_client.delete_dataset(f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}", delete_contents=True, not_found_ok=True)
bq_client.delete_dataset(f"{PROJECT_ID}.{RESULTS_DATASET_ID}", delete_contents=True, not_found_ok=True)

In [ ]:
# Delete Vertex AI resources

dataset.delete()
model.delete()
training_job.delete()
batch_predict_job.delete()

if os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_NAME